In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import matplotlib 
matplotlib.rcParams["figure.figsize"] = (20,10)

In [2]:
dataset = pd.read_csv('C:\\Users\\Lalita\\Downloads\\Bengaluru_House_Data.csv');

In [3]:
dataset.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00


In [4]:
re_data = dataset.drop(['area_type','society','availability'],axis='columns')
re_data.shape

(13320, 6)

In [5]:
re_data.isnull().sum()

location        1
size           16
total_sqft      0
bath           73
balcony       609
price           0
dtype: int64

In [6]:
df_clean = re_data.dropna()

In [7]:
df_clean.shape

(12710, 6)

In [8]:
df_clean['bhk']= df_clean['size'].apply(lambda x: int(x.split(' ')[0]))

<ipython-input-8-51f72178e4da>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['bhk']= df_clean['size'].apply(lambda x: int(x.split(' ')[0]))


In [9]:
df_clean["size"].unique()

array(['2 BHK', '4 Bedroom', '3 BHK', '3 Bedroom', '1 BHK', '1 RK',
       '4 BHK', '1 Bedroom', '2 Bedroom', '6 Bedroom', '8 Bedroom',
       '7 Bedroom', '5 BHK', '7 BHK', '6 BHK', '5 Bedroom', '11 BHK',
       '9 BHK', '9 Bedroom', '27 BHK', '11 Bedroom', '43 Bedroom',
       '14 BHK', '8 BHK', '12 Bedroom', '10 Bedroom', '13 BHK'],
      dtype=object)

In [10]:
def is_float(x):
    try:
        float(x)
    except:
        return False
    return True 

In [11]:
df_clean[~df_clean['total_sqft'].apply(is_float)]
print(df_clean.shape)

(12710, 7)


In [12]:
def convert_range_to_avarge(x):
    tokens = x.split('-')
    if len(tokens) == 2:
        return (float(tokens[0])+float(tokens[1]))/2
    try:
        return float(x)
    except:
        return None

In [13]:
df2 = df_clean.copy()
df2.total_sqft= df2['total_sqft'].apply(convert_range_to_avarge)
df2 = df2[df2.total_sqft.notnull()]
df2.head(2)

,location,size,total_sqft,bath,balcony,price,bhk
0,Electronic City Phase II,2 BHK,1056.0,2.0,1.0,39.07,2
1,Chikka Tirupathi,4 Bedroom,2600.0,5.0,3.0,120.00,4


In [14]:
df2.shape

(12668, 7)

In [15]:
len(df2['location'].unique())

1259

In [16]:
location_stats = df2.groupby('location')['location'].agg('count')
location_stats

location
 Anekal                 1
 Banaswadi              1
 Basavangudi            1
 Bhoganhalli            1
 Devarabeesana Halli    6
                       ..
singapura paradise      1
t.c palya               1
tc.palya                4
vinayakanagar           1
whitefiled              1
Name: location, Length: 1259, dtype: int64

In [17]:
location_stats_less_than_10= (location_stats[location_stats<10])

In [18]:
df2.location = df2.location.apply(lambda x:'other' if x in location_stats_less_than_10 else x)

In [19]:
df2.shape

(12668, 7)

In [20]:
df2['price_per_sqft'] = df2['price'] * 100000 / df2['total_sqft']

In [21]:
df2[df2.total_sqft/df2.bhk<300].head()


,location,size,total_sqft,bath,balcony,price,bhk,price_per_sqft
58,Murugeshpalya,6 Bedroom,1407.0,4.0,1.0,150.0,6,10660.980810
68,other,8 Bedroom,1350.0,7.0,0.0,85.0,8,6296.296296
70,other,3 Bedroom,500.0,3.0,2.0,100.0,3,20000.000000
78,Kaval Byrasandra,2 BHK,460.0,1.0,0.0,22.0,2,4782.608696
89,Rajaji Nagar,6 Bedroom,710.0,6.0,3.0,160.0,6,22535.211268


In [22]:
df3 =  df2[~((df2.total_sqft/df2.bhk)<300)]
print(df2.shape)
print(df3.shape)

(12668, 8)
(12013, 8)


In [23]:
df3.head()

,location,size,total_sqft,bath,balcony,price,bhk,price_per_sqft
0,Electronic City Phase II,2 BHK,1056.0,2.0,1.0,39.07,2,3699.810606
1,Chikka Tirupathi,4 Bedroom,2600.0,5.0,3.0,120.00,4,4615.384615
2,Uttarahalli,3 BHK,1440.0,2.0,3.0,62.00,3,4305.555556
3,Lingadheeranahalli,3 BHK,1521.0,3.0,1.0,95.00,3,6245.890861
4,Kothanur,2 BHK,1200.0,2.0,1.0,51.00,2,4250.000000


In [24]:
df3.isnull().sum()
#df3 = df3.dropna()

location          0
size              0
total_sqft        0
bath              0
balcony           0
price             0
bhk               0
price_per_sqft    0
dtype: int64

In [25]:
#remove outlier based on some thresold as per businuss requirement
df3.price_per_sqft.describe()

count     12013.000000
mean       6206.082347
std        3985.518807
min         267.829813
25%        4199.363057
50%        5252.525253
75%        6823.529412
max      176470.588235
Name: price_per_sqft, dtype: float64

In [26]:
def remove_outliers_per_location(df):
    df_out = pd.DataFrame()
    for key,value in df.groupby('location'):
        mean = np.mean(value.price_per_sqft)
        sd = np.std(value.price_per_sqft)
        reduced_df = value[(value.price_per_sqft<=(mean+1*sd))& (value.price_per_sqft>(mean-1*sd))]
        df_out = pd.concat([df_out,reduced_df],ignore_index= True)
    return df_out
df4 =    remove_outliers_per_location(df3)
df4.shape

(9828, 8)

In [27]:
df4.head(20)

,location,size,total_sqft,bath,balcony,price,bhk,price_per_sqft
0,Devarachikkanahalli,3 BHK,1250.0,2.0,3.0,44.0,3,3520.000000
1,Devarachikkanahalli,2 BHK,1250.0,2.0,2.0,40.0,2,3200.000000
2,Devarachikkanahalli,2 Bedroom,1200.0,2.0,2.0,83.0,2,6916.666667
3,Devarachikkanahalli,2 BHK,1170.0,2.0,2.0,40.0,2,3418.803419
4,Devarachikkanahalli,3 BHK,1425.0,2.0,2.0,65.0,3,4561.403509
5,Devarachikkanahalli,2 BHK,947.0,2.0,2.0,43.0,2,4540.654699
6,Devarachikkanahalli,2 BHK,1130.0,2.0,2.0,36.0,2,3185.840708
7,Devarachikkanahalli,3 BHK,1417.0,2.0,2.0,76.0,3,5363.443896
8,Devarachikkanahalli,2 BHK,1230.0,2.0,1.0,58.0,2,4715.447154
9,Devarachikkanahalli,3 BHK,1705.0,3.0,2.0,75.0,3,4398.826979


In [28]:
#outlier detetction 
#starting with boxplot
'''sns.boxplot(y='total_sqft', x='price_per_sqft',data=df3[['total_sqft','price_per_sqft']])
plt.xticks(rotation=90)
plt.ylabel('total_sqft')'''

"sns.boxplot(y='total_sqft', x='price_per_sqft',data=df3[['total_sqft','price_per_sqft']])\nplt.xticks(rotation=90)\nplt.ylabel('total_sqft')"

In [29]:
'''matplotlib.rcParams['figure.figsize'] = (15,10)
bhk2 = df4[(df4.location=='Devarachikkanahalli')&(df4.size==2)]
bhk3 = df4[(df4.location=='Devarachikkanahalli')&(df4.size==3)]
plt.scatter(bhk2.total_sqft, bhk2.price,color='blue',label='2',s=50)
plt.scatter(bhk3.total_sqft, bhk3.price,marker='+',color='green',label='3',s=50)
plt.xlabel("Total Square Feet Area")
plt.ylabel("Price (Lakh Indian Rupees)")
plt.title('Devarachikkanahalli')
plt.legend()'''

'matplotlib.rcParams[\'figure.figsize\'] = (15,10)\nbhk2 = df4[(df4.location==\'Devarachikkanahalli\')&(df4.size==2)]\nbhk3 = df4[(df4.location==\'Devarachikkanahalli\')&(df4.size==3)]\nplt.scatter(bhk2.total_sqft, bhk2.price,color=\'blue\',label=\'2\',s=50)\nplt.scatter(bhk3.total_sqft, bhk3.price,marker=\'+\',color=\'green\',label=\'3\',s=50)\nplt.xlabel("Total Square Feet Area")\nplt.ylabel("Price (Lakh Indian Rupees)")\nplt.title(\'Devarachikkanahalli\')\nplt.legend()'

In [30]:
def plot_scatter_chart(df,location):
    bhk2 = df[(df.location==location) & (df.size==2)]
    bhk3 = df[(df.location==location) & (df.size==3)]
    matplotlib.rcParams['figure.figsize'] = (15,10)
    plt.scatter(bhk2.total_sqft,bhk2.price,color='blue',label='2 BHK', s=50)
    plt.scatter(bhk3.total_sqft,bhk3.price,marker='+', color='green',label='3 BHK', s=50)
    plt.xlabel("Total Square Feet Area")
    plt.ylabel("Price (Lakh Indian Rupees)")
    plt.title(location)
    plt.legend()
    
#plot_scatter_chart(df7,"Rajaji Nagar")

In [31]:
def remove_bhk_outliers(df):
    exclude_indices = np.array([])
    for location, location_df in df.groupby('location'):
        bhk_stats = {}
        for bhk, bhk_df in location_df.groupby('bhk'):
            bhk_stats[bhk] = {
                'mean': np.mean(bhk_df.price_per_sqft),
                'std': np.std(bhk_df.price_per_sqft),
                'count': bhk_df.shape[0]
            }
        for bhk, bhk_df in location_df.groupby('bhk'):
            bhkint = int(bhk)-1
            stats = bhk_stats.get(bhkint)
            if stats and stats['count']>5:
                exclude_indices = np.append(exclude_indices, bhk_df[bhk_df.price_per_sqft<(stats['mean'])].index.values)
    return df.drop(exclude_indices,axis='index')
df5 = remove_bhk_outliers(df4)

In [32]:
#df6 = df5[df5.bath<df5.size+2]
df5.head()
df5.shape

(7070, 8)

In [33]:
df7 = df5.drop(['price_per_sqft','size'],axis='columns')
df7.head(3)

,location,total_sqft,bath,balcony,price,bhk
1,Devarachikkanahalli,1250.0,2.0,2.0,40.0,2
2,Devarachikkanahalli,1200.0,2.0,2.0,83.0,2
3,Devarachikkanahalli,1170.0,2.0,2.0,40.0,2


In [34]:
#apply one hot encoding 
dummieslocation = pd.get_dummies(df7.location)
#dummiesarea = pd.get_dummies(df7.area_type)

In [35]:
final_DF = pd.concat([df7, dummieslocation], axis = 1)
final_DF.columns

Index(['location', 'total_sqft', 'bath', 'balcony', 'price', 'bhk',
       ' Devarachikkanahalli', '1st Block Jayanagar', '1st Block Koramangala',
       '1st Phase JP Nagar',
       ...
       'Vishveshwarya Layout', 'Vishwapriya Layout', 'Vittasandra',
       'Whitefield', 'Yelachenahalli', 'Yelahanka', 'Yelahanka New Town',
       'Yelenahalli', 'Yeshwanthpur', 'other'],
      dtype='object', length=254)

In [36]:
final_df=final_DF.drop(['other'],axis='columns')

In [37]:
#final_df=final_df.drop(['Built-up  Area','Carpet  Area','Plot  Area'],axis='columns')

In [38]:
final_df=final_df.drop(['location'],axis='columns')
final_df.head()

,total_sqft,bath,balcony,price,bhk,Devarachikkanahalli,1st Block Jayanagar,1st Block Koramangala,1st Phase JP Nagar,2nd Phase Judicial Layout,...,Vijayanagar,Vishveshwarya Layout,Vishwapriya Layout,Vittasandra,Whitefield,Yelachenahalli,Yelahanka,Yelahanka New Town,Yelenahalli,Yeshwanthpur
1,1250.0,2.0,2.0,40.0,2,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1200.0,2.0,2.0,83.0,2,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1170.0,2.0,2.0,40.0,2,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1425.0,2.0,2.0,65.0,3,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,947.0,2.0,2.0,43.0,2,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
final_df.columns

Index(['total_sqft', 'bath', 'balcony', 'price', 'bhk', ' Devarachikkanahalli',
       '1st Block Jayanagar', '1st Block Koramangala', '1st Phase JP Nagar',
       '2nd Phase Judicial Layout',
       ...
       'Vijayanagar', 'Vishveshwarya Layout', 'Vishwapriya Layout',
       'Vittasandra', 'Whitefield', 'Yelachenahalli', 'Yelahanka',
       'Yelahanka New Town', 'Yelenahalli', 'Yeshwanthpur'],
      dtype='object', length=252)

In [40]:
#final_df=final_df.drop(['balcony'],axis='columns')

In [41]:
# remove independent variable
X = final_df.drop(['price'],axis='columns')
X.shape

(7070, 251)

In [42]:
Y = final_df.price
len(Y)

7070

In [43]:
from sklearn.model_selection  import train_test_split
Xtrain,Xtest,ytrain,ytest = train_test_split(X,Y,test_size=20,random_state=10)

In [44]:
from sklearn.linear_model import LinearRegression
lrmodel = LinearRegression()
lrmodel.fit(Xtrain,ytrain)
lrmodel.score(Xtest,ytest)

0.8742249830059533

In [45]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)

cross_val_score(LinearRegression(), X, Y, cv=cv)

array([0.85145894, 0.85674539, 0.77934183, 0.8778896 , 0.84196361])

In [46]:
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor

def find_best_model_using_gridsearchcv(X,y):
    algos = {
        'linear_regression' : {
            'model': LinearRegression(),
            'params': {
                'normalize': [True, False]
            }
        },
        'lasso': {
            'model': Lasso(),
            'params': {
                'alpha': [1,2],
                'selection': ['random', 'cyclic']
            }
        },
        'decision_tree': {
            'model': DecisionTreeRegressor(),
            'params': {
                'criterion' : ['mse','friedman_mse'],
                'splitter': ['best','random']
            }
        }
    }
    scores = []
    cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
    for algo_name, config in algos.items():
        gs =  GridSearchCV(config['model'], config['params'], cv=cv, return_train_score=False)
        gs.fit(X,y)
        scores.append({
            'model': algo_name,
            'best_score': gs.best_score_,
            'best_params': gs.best_params_
        })

    return pd.DataFrame(scores,columns=['model','best_score','best_params'])

find_best_model_using_gridsearchcv(X,Y)

,model,best_score,best_params
0,linear_regression,0.84148,{'normalize': False}
1,lasso,0.69550,"{'alpha': 1, 'selection': 'random'}"
2,decision_tree,0.73065,"{'criterion': 'friedman_mse', 'splitter': 'best'}"


In [48]:
def predict_price(location,sqft,bath,bhk,balcony):    
    loc_index = np.where(X.columns==location)[0][0]

    x = np.zeros(len(X.columns))
    x[0] = sqft
    x[1] = bath
    x[2] = balcony
    x[3] = bhk
    if loc_index >= 0:
        x[loc_index] = 1

    return lrmodel.predict([x])[0]

In [65]:
predict_price('Indira Nagar',1000, 2, 2,2)

157.62078175181614

In [67]:
import pickle
with open('banglore_home_prices_model.pickle','wb') as f:
    pickle.dump(lrmodel,f)

In [68]:
import json
columns = {
    'data_columns' : [col.lower() for col in X.columns]
}
with open("columns.json","w") as f:
    f.write(json.dumps(columns))